In [1]:
pwd

'D:\\ML\\EVA\\JEDI\\Captsone_EVA5'

In [4]:
os.sep

'\\'

In [2]:
from model import *
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
model = fork(depth_freeze = False, yolo_freeze = True,depth_preload_pth = 'D:/ML/EVA/JEDi/Midas/model-f46da743.pt', yolo_preload_pth = 'D:/ML/EVA/JEDI/YoloV3master/last_ppe.pt').to(device)
from torchsummary import summary
print(summary(model, (3,416,416)))

Model Summary: 225 layers, 6.25895e+07 parameters, 6.25895e+07 gradients


Using cache found in C:\Users\realp/.cache\torch\hub\facebookresearch_WSL-Images_master


encoder loaded weights from D:/ML/EVA/JEDi/Midas/model-f46da743.pt
resnet_layer 0 was frozen
resnet_layer 1 was frozen
resnet_layer 2 was frozen
resnet_layer 3 was frozen
depth_decoder loaded from D:/ML/EVA/JEDi/Midas/model-f46da743.pt
loaded weight from D:/ML/EVA/JEDI/YoloV3master/last_ppe.pt
yolo_decoder loaded from D:/ML/EVA/JEDI/YoloV3master/last_ppe.pt
yolo_layer 0 was frozen
yolo_layer 1 was frozen
yolo_layer 2 was frozen
yolo_layer 3 was frozen
yolo_layer 4 was frozen
yolo_layer 5 was frozen
yolo_layer 6 was frozen
yolo_layer 7 was frozen
yolo_layer 8 was frozen
yolo_layer 9 was frozen
yolo_layer 10 was frozen
yolo_layer 11 was frozen
yolo_layer 12 was frozen
----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 64, 208, 208]           9,408
       BatchNorm2d-2         [-1, 64, 208, 208]             128
              ReLU-3         [-1, 64, 208, 208]               0
    

In [1]:
import torch.optim as optim
import torch.optim.lr_scheduler as lr_scheduler
import os
from model import *

Model Summary: 225 layers, 6.25895e+07 parameters, 6.25895e+07 gradients


In [2]:
wdir = 'weights' + os.sep  # weights dir
last = wdir + 'last.pt'
best = wdir + 'best.pt'
results_file = 'results.txt'

In [3]:
hyp = {'giou': 3.54,  # giou loss gain
       'cls': 37.4,  # cls loss gain
       'cls_pw': 1.0,  # cls BCELoss positive_weight
       'obj': 64.3,  # obj loss gain (*=img_size/320 if img_size != 320)
       'obj_pw': 1.0,  # obj BCELoss positive_weight
       'iou_t': 0.225,  # iou training threshold
       'lr0': 0.01,  # initial learning rate (SGD=5E-3, Adam=5E-4)
       'lrf': 0.0005,  # final learning rate (with cos scheduler)
       'momentum': 0.937,  # SGD momentum
       'weight_decay': 0.000484,  # optimizer weight decay
       'fl_gamma': 0.0,  # focal loss gamma (efficientDet default is gamma=1.5)
       'hsv_h': 0.0138,  # image HSV-Hue augmentation (fraction)
       'hsv_s': 0.678,  # image HSV-Saturation augmentation (fraction)
       'hsv_v': 0.36,  # image HSV-Value augmentation (fraction)
       'degrees': 1.98 * 0,  # image rotation (+/- deg)
       'translate': 0.05 * 0,  # image translation (+/- fraction)
       'scale': 0.05 * 0,  # image scale (+/- gain)
       'shear': 0.641 * 0}  # image shear (+/- deg)

In [21]:
from types import SimpleNamespace   
d = dict(data='D:/ML/EVA/YoloV3/data/customdata/custom.data',
epochs = 1, 
batch_size = 10, 
multi_scale = False,
cache_images = True,
img_size = [512, 512, 512], 
accumulate = 4,
single_cls = False,
adam = False,
rect = False
        )
opt = SimpleNamespace(**d)

In [5]:
# train 
data = opt.data
epochs = opt.epochs
batch_size = opt.batch_size
accumulate = opt.accumulate  # effective bs = batch_size * accumulate = 16 * 4 = 64
#weights = opt.weights  # initial training weights
imgsz_min, imgsz_max, imgsz_test = opt.img_size

#
init_seeds()

In [6]:
data_dict = parse_data_cfg(data)
train_path = data_dict['train']
test_path = data_dict['valid']
nc = 1 if opt.single_cls else int(data_dict['classes'])  # number of classes
hyp['cls'] *= nc / 80  # update coco-tuned hyp['cls'] to current dataset
for f in glob.glob('*_batch*.png') + glob.glob(results_file):
    os.remove(f)

In [7]:
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
model = fork(depth_freeze = True, yolo_freeze = False,depth_preload_pth = '', yolo_preload_pth = 'D:/ML/EVA/JEDI/YoloV3master/last_ppe.pt').to(device)

Using cache found in C:\Users\realp/.cache\torch\hub\facebookresearch_WSL-Images_master


encoder loaded weights from D:/ML/EVA/JEDi/Midas/model-f46da743.pt
resnet_layer 0 was frozen
resnet_layer 1 was frozen
resnet_layer 2 was frozen
resnet_layer 3 was frozen
depth_layer 0 was frozen
loaded weight from D:/ML/EVA/JEDI/YoloV3master/last_ppe.pt
yolo_decoder loaded from D:/ML/EVA/JEDI/YoloV3master/last_ppe.pt


In [17]:
trainables = [name for name, param in model.named_parameters() if param.requires_grad]
#list(model.named_parameters())
#len(trainables)
trainables

['yolo_decoder.yolo_13_bottle_neck.weight',
 'yolo_decoder.yolo_13_path.Conv2d_0.weight',
 'yolo_decoder.yolo_13_path.BatchNorm2d_1.weight',
 'yolo_decoder.yolo_13_path.BatchNorm2d_1.bias',
 'yolo_decoder.yolo_13_path.Conv2d_3.weight',
 'yolo_decoder.yolo_13_path.BatchNorm2d_4.weight',
 'yolo_decoder.yolo_13_path.BatchNorm2d_4.bias',
 'yolo_decoder.yolo_13_path.Conv2d_6.weight',
 'yolo_decoder.yolo_13_path.BatchNorm2d_7.weight',
 'yolo_decoder.yolo_13_path.BatchNorm2d_7.bias',
 'yolo_decoder.yolo_13_tail.Conv2d_0.weight',
 'yolo_decoder.yolo_13_tail.BatchNorm2d_1.weight',
 'yolo_decoder.yolo_13_tail.BatchNorm2d_1.bias',
 'yolo_decoder.yolo_13_tail.Conv2d_3.weight',
 'yolo_decoder.yolo_13_tail.Conv2d_3.bias',
 'yolo_decoder.yolo_26_upsample.Conv2d_0.weight',
 'yolo_decoder.yolo_26_upsample.BatchNorm2d_1.weight',
 'yolo_decoder.yolo_26_upsample.BatchNorm2d_1.bias',
 'yolo_decoder.yolo_26_bottle_neck.weight',
 'yolo_decoder.yolo_26_path.Conv2d_0.weight',
 'yolo_decoder.yolo_26_path.BatchN

In [8]:
pg0, pg1, pg2 = [], [], []  # optimizer parameter groups
for k, v in dict(model.named_parameters()).items():
    if v.requires_grad: 
        if '.bias' in k:
            pg2 += [v]  # biases
        elif '.weight' in k and 'Conv2d' in k:
            pg1 += [v]  # apply weight_decay
        else:
            pg0 += [v]  # all else
#len(pg2) + len(pg1) + len(pg0), len(pg2),len(pg1),len(pg0)

In [9]:
if opt.adam:
    # hyp['lr0'] *= 0.1  # reduce lr (i.e. SGD=5E-3, Adam=5E-4)
    optimizer = optim.Adam(pg0, lr=hyp['lr0'])
    # optimizer = AdaBound(pg0, lr=hyp['lr0'], final_lr=0.1)
else:
    optimizer = optim.SGD(pg0, lr=hyp['lr0'], momentum=hyp['momentum'], nesterov=True)
optimizer.add_param_group({'params': pg1, 'weight_decay': hyp['weight_decay']})  # add pg1 with weight_decay
optimizer.add_param_group({'params': pg2})  # add pg2 (biases)
del pg0, pg1, pg2

In [10]:
#mixed precision
start_epoch = 0
best_fitness = 0.0
mixed_precision = True
try:  # Mixed precision training https://github.com/NVIDIA/apex
    from apex import amp
except:
    # print('Apex recommended for mixed precision and faster training: https://github.com/NVIDIA/apex')
    mixed_precision = False
if mixed_precision:
    model, optimizer = amp.initialize(model, optimizer, opt_level='O1', verbosity=0)


In [11]:
#scheduler
lf = lambda x: (((1 + math.cos(
    x * math.pi / epochs)) / 2) ** 1.0) * 0.95 + 0.05  # cosine https://arxiv.org/pdf/1812.01187.pdf
scheduler = lr_scheduler.LambdaLR(optimizer, lr_lambda=lf, last_epoch=start_epoch - 1)

In [19]:
train_path, batch_size

('data/customdata/train.txt', 10)

In [ ]:
from utils_yolo.datasets import * 

In [23]:
cd D:\ML\EVA\YoloV3

D:\ML\EVA\YoloV3


In [25]:
dataset = LoadImagesAndLabels(train_path, img_size = 416 , batch_size = batch_size,
                                  augment=True,
                                  hyp=hyp,  # augmentation hyperparameters
                                  rect=opt.rect,  # rectangular training
                                  cache_images=opt.cache_images,
                                  single_cls=opt.single_cls)

Caching labels (3064 found, 139 missing, 0 empty, 0 duplicate, for 3203 images): 100%|█| 3203/3203 [00:00<00:00, 5651.1
Caching images (1.2GB): 100%|█████████████████████████████████████████████████████| 3203/3203 [00:22<00:00, 142.82it/s]


In [57]:
dataset[0][0].size(),dataset[0][1].size(),dataset[0][2],dataset[0][3] 

(torch.Size([3, 416, 416]),
 torch.Size([1, 6]),
 'data\\customdata\\images\\01-prince-boots - Copy.jpg',
 None)

In [59]:
batch_size = min(batch_size, len(dataset))
nw = min([os.cpu_count(), batch_size if batch_size > 1 else 0, 8])  # number of workers
dataloader = torch.utils.data.DataLoader(dataset,
                                         batch_size=batch_size,
                                         num_workers=nw,
                                         shuffle=not opt.rect,  # Shuffle=True unless rectangular training is used
                                         pin_memory=True,
                                         collate_fn=dataset.collate_fn)

In [62]:
a, b, c, d = next(iter(dataloader))

MemoryError: 

In [28]:
testloader = torch.utils.data.DataLoader(LoadImagesAndLabels(test_path, imgsz_test, batch_size,
                                                             hyp=hyp,
                                                             rect=True,
                                                             cache_images=opt.cache_images,
                                                             single_cls=opt.single_cls),
                                         batch_size=batch_size,
                                         num_workers=nw,
                                         pin_memory=True,
                                         collate_fn=dataset.collate_fn)

Caching labels (285 found, 1 missing, 0 empty, 0 duplicate, for 286 images): 100%|█| 286/286 [00:00<00:00, 1570.79it/s]
Caching images (0.1GB): 100%|████████████████████████████████████████████████████████| 286/286 [00:03<00:00, 89.97it/s]


In [29]:
# Model parameters
model.nc = nc  # attach number of classes to model
model.hyp = hyp  # attach hyperparameters to model
model.gr = 1.0  # giou loss ratio (obj_loss = 1.0 or giou)
model.class_weights = labels_to_class_weights(dataset.labels, nc).to(device)  # attach class weights

# Model EMA
ema = torch_utils.ModelEMA(model)


In [42]:
k = next(iter(dataloader))

MemoryError: 

In [38]:
# Start training
nb = len(dataloader)  # number of batches
n_burn = max(3 * nb, 500)  # burn-in iterations, max(3 epochs, 500 iterations)
maps = np.zeros(nc)  # mAP per class
# torch.autograd.set_detect_anomaly(True)
results = (0, 0, 0, 0, 0, 0, 0)  # 'P', 'R', 'mAP', 'F1', 'val GIoU', 'val Objectness', 'val Classification'
t0 = time.time()
print('Image sizes %g - %g train, %g test' % (imgsz_min, imgsz_max, imgsz_test))
print('Using %g dataloader workers' % nw)
print('Starting training for %g epochs...' % epochs)
for epoch in range(start_epoch, epochs):# epoch ------------------------------------------------------------------
    model.train()
    mloss = torch.zeros(4).to(device)  # mean losses
    print(('\n' + '%10s' * 8) % ('Epoch', 'gpu_mem', 'GIoU', 'obj', 'cls', 'total', 'targets', 'img_size'))
    pbar = tqdm(enumerate(dataloader), total=nb)  # progress bar
    for i, (imgs, targets, paths, _) in pbar:  # batch -------------------------------------------------------------

        # Run model
        #pred = model.forward(imgs)
        print(imgs.size)
        # Compute loss
        #loss, loss_items = compute_loss(pred, targets, model)
        #print(loss)
#         if not torch.isfinite(loss):
#             print('WARNING: non-finite loss, ending training ', loss_items)
#             return results

        # Scale loss by nominal batch_size of 64
        #loss *= batch_size / 64
        #print(loss)
        break 

Image sizes 512 - 512 train, 512 test
Using 8 dataloader workers
Starting training for 1 epochs...

     Epoch   gpu_mem      GIoU       obj       cls     total   targets  img_size


MemoryError: 